# 📊 Camada Bronze - Visualização de Dados Brutos

Este notebook demonstra como visualizar e consultar dados da camada Bronze usando Spark.

## O que é a Camada Bronze?
- Dados brutos coletados diretamente das APIs
- Sem transformações ou limpeza
- Formato Parquet armazenado no MinIO
- Particionado por data de ingestão

In [ ]:
# ⚠️ IMPORTANTE: Execute primeiro o notebook CONFIGURAR_SPARK.ipynb
# Se ainda não executou, execute o script de correção:
try:
    spark
    print("✅ Spark Session já disponível")
except NameError:
    print("⚠️  Spark não configurado. Execute CONFIGURAR_SPARK.ipynb primeiro!")
    print("Ou execute o script de correção:")
    exec(open('/home/jovyan/work/fix_spark_py4j.py').read())

In [ ]:
# Verificar Spark Session
print(f"✅ Spark versão: {spark.version}")
print(f"✅ App: {spark.sparkContext.appName}")

## 1. Listar Arquivos na Camada Bronze

In [ ]:
# Listar todos os arquivos Bronze disponíveis
from pyspark.sql.functions import col

try:
    # Listar arquivos no MinIO
    bronze_paths = [
        "s3a://govbr/bronze/ibge/municipios/",
        "s3a://govbr/bronze/ibge/estados/",
        "s3a://govbr/bronze/ibge/populacao_estados/",
        "s3a://govbr/bronze/portal_transparencia/orgaos_siafi/",
        "s3a://govbr/bronze/portal_transparencia/bpc_municipios/"
    ]
    
    print("📁 Arquivos disponíveis na Camada Bronze:\n")
    for path in bronze_paths:
        try:
            df = spark.read.parquet(path)
            print(f"✅ {path}")
            print(f"   Registros: {df.count():,}")
            print(f"   Colunas: {', '.join(df.columns[:5])}...")
            print()
        except Exception as e:
            print(f"⚠️  {path} - Não disponível")
            print()
except Exception as e:
    print(f"❌ Erro: {e}")

## 2. Visualizar Municípios (IBGE)

In [ ]:
# Ler dados de municípios
df_municipios = spark.read.parquet("s3a://govbr/bronze/ibge/municipios/")

print(f"📊 Total de municípios: {df_municipios.count():,}")
print(f"\n📋 Estrutura dos dados:")
df_municipios.printSchema()

print(f"\n🔍 Primeiros registros:")
df_municipios.show(10, truncate=False)

In [ ]:
# Análise por região
from pyspark.sql.functions import count, col

municipios_por_regiao = df_municipios \
    .groupBy("regiao_nome") \
    .agg(count("*").alias("total_municipios")) \
    .orderBy(col("total_municipios").desc())

print("📊 Municípios por Região:")
municipios_por_regiao.show(truncate=False)

## 3. Visualizar Estados (IBGE)

In [ ]:
# Ler dados de estados
df_estados = spark.read.parquet("s3a://govbr/bronze/ibge/estados/")

print(f"📊 Total de estados: {df_estados.count()}")
print(f"\n🔍 Todos os estados:")
df_estados.select("uf_sigla", "uf_nome", "regiao_nome").show(truncate=False)

## 4. Visualizar Dados de BPC (Portal da Transparência)

In [ ]:
# Ler dados de BPC
try:
    df_bpc = spark.read.parquet("s3a://govbr/bronze/portal_transparencia/bpc_municipios/")
    
    print(f"📊 Total de registros BPC: {df_bpc.count():,}")
    print(f"\n📋 Estrutura:")
    df_bpc.printSchema()
    
    print(f"\n🔍 Primeiros registros:")
    df_bpc.show(10, truncate=False)
    
    # Valor total por estado
    from pyspark.sql.functions import sum, col
    
    bpc_por_estado = df_bpc \
        .groupBy("uf_sigla") \
        .agg(
            sum("valor").alias("valor_total"),
            sum("quantidade_beneficiados").alias("total_beneficiados")
        ) \
        .orderBy(col("valor_total").desc())
    
    print(f"\n💰 BPC por Estado:")
    bpc_por_estado.show(truncate=False)
    
except Exception as e:
    print(f"⚠️  Dados de BPC não disponíveis: {e}")

## 5. Consultas SQL na Camada Bronze

In [ ]:
# Criar views temporárias
df_municipios.createOrReplaceTempView("municipios")
df_estados.createOrReplaceTempView("estados")

# Consulta: Municípios por estado
resultado = spark.sql("""
    SELECT 
        e.uf_sigla,
        e.uf_nome,
        COUNT(m.codigo_ibge) as total_municipios
    FROM estados e
    LEFT JOIN municipios m ON e.uf_sigla = m.uf_sigla
    GROUP BY e.uf_sigla, e.uf_nome
    ORDER BY total_municipios DESC
    LIMIT 10
""")

print("📊 Top 10 Estados por Número de Municípios:")
resultado.show(truncate=False)

## ✅ Resumo

A camada Bronze contém dados brutos coletados diretamente das APIs:
- ✅ IBGE: Municípios, Estados, População
- ✅ Portal da Transparência: Órgãos SIAFI, BPC
- ✅ Formato: Parquet particionado por data
- ✅ Armazenamento: MinIO (S3-compatible)